In [ ]:
#!/usr/bin/env python3
"""
Classificateur médical avec analyse sémantique améliorée
Format simple - sans classes ni dépendances externes
"""

import os
import shutil
import json
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Configuration
BASE_DIR = os.path.abspath(r"C:\Users\kosmo\pycode\Iqvia_process")
CHEMIN_SOURCE = os.path.join(BASE_DIR, "ProcessEx")
CHEMIN_DEPOTS = os.path.join(BASE_DIR, "Depots")

# Initialiser le LLM
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️ ATTENTION: Clé API non trouvée dans les variables d'environnement")
    api_key = "votre_clé_api_openai_ici"  # À remplacer par votre vraie clé

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,  # Légère créativité pour nuances sémantiques
    api_key=api_key
)

def detection_mots_cles_medicaux(titre: str) -> dict:
    """Détection fallback par mots-clés enrichie"""
    titre_lower = titre.lower()
    
    # Dictionnaire enrichi avec contextes sémantiques
    termes_contextes = {
        # Pathologies oncologiques
        'cancer': ('oncologie', 'clinique'), 'tumeur': ('oncologie', 'clinique'), 
        'oncologie': ('oncologie', 'clinique'), 'prostate': ('oncologie', 'clinique'),
        'sein': ('oncologie', 'clinique'), 'poumon': ('oncologie', 'clinique'),
        'métastase': ('oncologie', 'clinique'), 'chimiothérapie': ('oncologie', 'traitement'),
        
        # Pathologies endocriniennes
        'diabète': ('diabetologie', 'clinique'), 'diabete': ('diabetologie', 'clinique'), 
        'glycémie': ('diabetologie', 'clinique'), 'insuline': ('diabetologie', 'traitement'),
        'hba1c': ('diabetologie', 'diagnostic'),
        
        # Pathologies cardiovasculaires
        'cardiaque': ('cardiologie', 'clinique'), 'coeur': ('cardiologie', 'clinique'), 
        'cœur': ('cardiologie', 'clinique'), 'hypertension': ('cardiologie', 'clinique'),
        'infarctus': ('cardiologie', 'urgence'), 'arythmie': ('cardiologie', 'clinique'),
        
        # Pathologies neurologiques
        'alzheimer': ('neurologie', 'clinique'), 'parkinson': ('neurologie', 'clinique'), 
        'neurologie': ('neurologie', 'clinique'), 'épilepsie': ('neurologie', 'clinique'),
        'avc': ('neurologie', 'urgence'), 'sclérose': ('neurologie', 'clinique'),
        
        # Pathologies respiratoires
        'asthme': ('pneumologie', 'clinique'), 'pneumonie': ('pneumologie', 'urgence'), 
        'bronchite': ('pneumologie', 'clinique'), 'bpco': ('pneumologie', 'clinique'),
        
        # Pathologies dermatologiques
        'eczéma': ('dermatologie', 'clinique'), 'eczema': ('dermatologie', 'clinique'), 
        'psoriasis': ('dermatologie', 'clinique'), 'dermatite': ('dermatologie', 'clinique'),
        
        # Contextes non-pathologiques
        'formation': ('formation', 'commercial'), 'training': ('formation', 'commercial'),
        'vente': ('commercial', 'business'), 'marketing': ('commercial', 'business'),
        'étude': ('recherche', 'scientifique'), 'essai': ('recherche', 'scientifique'),
        'protocole': ('recherche', 'scientifique'), 'phase': ('recherche', 'scientifique'),
        'réglementation': ('reglementaire', 'administratif'), 'autorisation': ('reglementaire', 'administratif'),
        'surveillance': ('pharmacovigilance', 'securite'), 'observance': ('pharmacovigilance', 'therapeutique')
    }
    
    termes_detectes = []
    contextes_detectes = []
    
    for terme, (domaine, contexte) in termes_contextes.items():
        if terme in titre_lower:
            termes_detectes.append(terme)
            contextes_detectes.append((domaine, contexte))
    
    if termes_detectes:
        # Prendre le premier domaine détecté
        domaine_principal, contexte_principal = contextes_detectes[0]
        confiance = "haute" if len(termes_detectes) > 1 else "moyenne"
        
        # Déterminer si c'est médical
        domaines_medicaux = ['oncologie', 'diabetologie', 'cardiologie', 'neurologie', 'pneumologie', 'dermatologie']
        est_medical = domaine_principal in domaines_medicaux
        
        return {
            "contient_maladie": est_medical,
            "maladies_detectees": termes_detectes,
            "categorie_medicale": domaine_principal,
            "contexte_principal": contexte_principal,
            "confiance": confiance,
            "titre_normalise": titre,
            "score_semantique": 0.7 if confiance == "haute" else 0.5
        }
    
    return {
        "contient_maladie": False,
        "maladies_detectees": [],
        "categorie_medicale": "aucune",
        "contexte_principal": "autre",
        "confiance": "faible",
        "titre_normalise": titre,
        "score_semantique": 0.2
    }

def analyser_titre_avec_llm_semantique(titre: str) -> dict:
    """Analyse sémantique avancée avec LLM - prompt enrichi"""
    try:
        analysis_prompt_template = ChatPromptTemplate.from_messages([
            ("system", """Tu es un expert en analyse sémantique médicale et business pharmaceutique. 
            Ton rôle est de comprendre le CONTEXTE et l'INTENTION derrière chaque titre, pas seulement chercher des mots-clés.
            
            Tu maîtrises ces domaines :
            - CLINIQUE : pathologies, diagnostics, traitements, soins patients
            - RECHERCHE : études, essais, protocoles, développement
            - COMMERCIAL : formation équipes, marketing, ventes, business
            - RÉGLEMENTAIRE : autorisations, compliance, pharmacovigilance
            - FORMATION : training, éducation, guidelines
            """),
            ("human", """
Analyse sémantique COMPLÈTE du titre : "{titre}"

Ne te contente pas de chercher des mots-clés ! Analyse le CONTEXTE :

EXEMPLES D'ANALYSE SÉMANTIQUE :
- "Formation équipe vente oncologie Q1 2025" → CONTEXTE=commercial, DOMAINE=oncologie, TYPE=formation
- "Suivi patients diabète hôpital" → CONTEXTE=clinique, DOMAINE=diabetologie, TYPE=suivi_medical  
- "Protocole étude phase III cancer poumon" → CONTEXTE=recherche, DOMAINE=oncologie, TYPE=essai_clinique
- "Rapport surveillance post-marketing" → CONTEXTE=reglementaire, DOMAINE=pharmacovigilance, TYPE=surveillance
- "Présentation résultats Q4 région" → CONTEXTE=commercial, DOMAINE=business, TYPE=reporting

MISSION : Identifie le CONTEXTE principal, le DOMAINE concerné, et la FINALITÉ du document.

Réponds UNIQUEMENT en JSON valide :
{{
    "contexte_principal": "clinique/recherche/commercial/reglementaire/formation/autre",
    "domaine_medical": "oncologie/cardiologie/neurologie/diabetologie/pneumologie/dermatologie/pharmacovigilance/business/aucun",
    "type_document": "formation/etude/rapport/suivi/presentation/protocole/guide/autre",
    "population_cible": "patients/professionnels/equipes_vente/chercheurs/regulateurs/mixte",
    "maladies_detectees": ["terme1", "terme2"],
    "contient_maladie": true/false,
    "confiance": "haute/moyenne/faible",
    "score_semantique": 0.8,
    "justification": "Explication courte du contexte détecté",
    "titre_normalise": "{titre}"
}}
            """)
        ])
        
        prompt_value = analysis_prompt_template.invoke({"titre": titre})
        response = llm.invoke(prompt_value.to_messages())
        content = response.content.strip()
        
        # Nettoyer le JSON
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0]
        elif "```" in content:
            content = content.split("```")[1].split("```")[0]
        
        resultat = json.loads(content.strip())
        
        # Validation et enrichissement
        if "categorie_medicale" not in resultat:
            resultat["categorie_medicale"] = resultat.get("domaine_medical", "aucune")
        
        # Calculer score sémantique si absent
        if "score_semantique" not in resultat or not isinstance(resultat["score_semantique"], (int, float)):
            contexte = resultat.get("contexte_principal", "autre")
            domaine = resultat.get("domaine_medical", "aucun")
            
            score = 0.3  # Base
            if contexte != "autre": score += 0.2
            if domaine != "aucun": score += 0.2
            if len(resultat.get("maladies_detectees", [])) > 0: score += 0.2
            if resultat.get("confiance") == "haute": score += 0.1
            
            resultat["score_semantique"] = min(0.95, score)
        
        return resultat
        
    except Exception as e:
        print(f"❌ Erreur LLM sémantique pour '{titre[:30]}...': {e}")
        return detection_mots_cles_medicaux(titre)

def rechercher_fichiers_filtres():
    """Phase 1: Recherche et filtre les fichiers par extension, date et taille"""
    print("🔍 PHASE 1: Recherche et filtrage des fichiers...")
    print("-" * 50)
    
    if not os.path.exists(CHEMIN_SOURCE):
        print(f"❌ Erreur: {CHEMIN_SOURCE} n'existe pas")
        return []
    
    # Définition des filtres
    DATE_LIMITE = datetime.now() - timedelta(days=365)  # Fichiers de moins d'un an
    TAILLE_MIN_OCTETS = 1 * 1024         # 1 KB minimum (pour éviter fichiers vides)
    TAILLE_MAX_OCTETS = 50 * 1024 * 1024  # 50 MB maximum
    TAILLE_CLASSIFICATION = 3000 * 1024   # 2 MB - Seuil pour classification sémantique
    
    print(f"📅 Filtre date: fichiers modifiés après le {DATE_LIMITE.strftime('%d/%m/%Y')}")
    print(f"📏 Filtre taille: entre {TAILLE_MIN_OCTETS//1024} KB et {TAILLE_MAX_OCTETS//1024//1024} MB")
    print(f"🧠 Classification sémantique: fichiers > {TAILLE_CLASSIFICATION//1024} KB seulement")
    print(f"📁 Fichiers ≤ {TAILLE_CLASSIFICATION//1024} KB → dossier AUTRES automatiquement")
    print()
    
    fichiers_trouves = []
    dossiers_stats = {}
    stats_filtrage = {
        'total_examines': 0,
        'rejetes_extension': 0,
        'rejetes_taille': 0, 
        'rejetes_date': 0,
        'rejetes_acces': 0,
        'acceptes': 0
    }
    
    for item in os.listdir(CHEMIN_SOURCE):
        item_path = os.path.join(CHEMIN_SOURCE, item)
        if os.path.isdir(item_path):
            print(f"📂 Analyse du dossier: {item}")
            count_dossier = 0
            
            for root, dirs, files in os.walk(item_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    stats_filtrage['total_examines'] += 1
                    
                    # Vérification de l'extension
                    if not file.lower().endswith(('.ppt', '.pptx', '.pdf')):
                        stats_filtrage['rejetes_extension'] += 1
                        continue
                    
                    try:
                        # Récupération des métadonnées
                        taille = os.path.getsize(file_path)
                        date_mod = datetime.fromtimestamp(os.path.getmtime(file_path))
                        
                        # Filtrage par taille
                        if not (TAILLE_MIN_OCTETS <= taille <= TAILLE_MAX_OCTETS):
                            stats_filtrage['rejetes_taille'] += 1
                            continue
                        
                        # Filtrage par date  
                        if date_mod < DATE_LIMITE:
                            stats_filtrage['rejetes_date'] += 1
                            continue
                        
                        # Fichier accepté
                        fichier_info = {
                            'chemin': file_path,
                            'nom': file,
                            'taille': taille,
                            'date_modification': date_mod,
                            'taille_mb': round(taille / (1024*1024), 2),
                            'taille_kb': round(taille / 1024, 1),
                            'age_jours': (datetime.now() - date_mod).days,
                            'classification_semantique': taille >= TAILLE_CLASSIFICATION  # True si > 2MB
                        }
                        
                        fichiers_trouves.append(fichier_info)
                        count_dossier += 1
                        stats_filtrage['acceptes'] += 1
                        
                    except (OSError, PermissionError) as e:
                        stats_filtrage['rejetes_acces'] += 1
                        print(f"   ⚠️ Ignoré (accès refusé): {os.path.basename(file_path)}")
            
            dossiers_stats[item] = count_dossier
            print(f"   ✅ {count_dossier} fichier(s) retenu(s) après filtrage")
    
    # Affichage des statistiques détaillées
    print(f"\n📊 STATISTIQUES DE FILTRAGE:")
    print(f"   • Fichiers examinés: {stats_filtrage['total_examines']}")
    print(f"   • ❌ Rejetés - extension: {stats_filtrage['rejetes_extension']}")
    print(f"   • ❌ Rejetés - taille: {stats_filtrage['rejetes_taille']}")
    print(f"   • ❌ Rejetés - date: {stats_filtrage['rejetes_date']}")
    print(f"   • ❌ Rejetés - accès: {stats_filtrage['rejetes_acces']}")
    print(f"   • ✅ RETENUS: {stats_filtrage['acceptes']}")
    
    print(f"\n📁 RÉPARTITION PAR DOSSIER:")
    for dossier, count in dossiers_stats.items():
        print(f"   • {dossier}: {count} fichier(s)")
    
    return fichiers_trouves

def trier_fichiers_par_criteres(fichiers_info):
    """Trie les fichiers par différents critères et affiche des statistiques"""
    print(f"\n📈 PHASE 1.5: Analyse et tri des fichiers retenus...")
    print("-" * 50)
    
    if not fichiers_info:
        print("❌ Aucun fichier à analyser")
        return fichiers_info
    
    # Tri par taille (décroissant)
    fichiers_par_taille = sorted(fichiers_info, key=lambda x: x['taille'], reverse=True)
    print(f"📏 Top 5 fichiers les plus volumineux:")
    for i, f in enumerate(fichiers_par_taille[:5], 1):
        print(f"   {i}. {f['nom'][:50]}... ({f['taille_mb']} MB)")
    
    # Tri par date (plus récent d'abord) 
    fichiers_par_date = sorted(fichiers_info, key=lambda x: x['date_modification'], reverse=True)
    print(f"\n📅 Top 5 fichiers les plus récents:")
    for i, f in enumerate(fichiers_par_date[:5], 1):
        date_str = f['date_modification'].strftime('%d/%m/%Y')
        print(f"   {i}. {f['nom'][:50]}... ({date_str}, {f['age_jours']} jours)")
    
    # Statistiques de répartition
    tailles = [f['taille_mb'] for f in fichiers_info]
    ages = [f['age_jours'] for f in fichiers_info]
    
    print(f"\n📊 STATISTIQUES DESCRIPTIVES:")
    print(f"   Taille - Moyenne: {sum(tailles)/len(tailles):.1f} MB, Max: {max(tailles):.1f} MB, Min: {min(tailles):.1f} MB")
    print(f"   Âge - Moyenne: {sum(ages)//len(ages)} jours, Max: {max(ages)} jours, Min: {min(ages)} jours")
    
    # Répartition par tranches de taille et éligibilité classification
    tranches_taille = {'< 1MB': 0, '1-2MB': 0, '2-5MB': 0, '5-20MB': 0, '> 20MB': 0}
    eligibles_classification = 0
    
    for f in fichiers_info:
        mb = f['taille_mb']
        if mb < 1: tranches_taille['< 1MB'] += 1
        elif mb < 2: tranches_taille['1-2MB'] += 1  
        elif mb < 5: tranches_taille['2-5MB'] += 1
        elif mb < 20: tranches_taille['5-20MB'] += 1
        else: tranches_taille['> 20MB'] += 1
        
        if f['classification_semantique']:
            eligibles_classification += 1
    
    print(f"\n📏 RÉPARTITION PAR TAILLE:")
    for tranche, count in tranches_taille.items():
        pct = (count / len(fichiers_info) * 100) if fichiers_info else 0
        print(f"   • {tranche}: {count} fichiers ({pct:.1f}%)")
    
    print(f"\n🧠 ÉLIGIBILITÉ CLASSIFICATION SÉMANTIQUE:")
    print(f"   • Fichiers > 2MB (analysés): {eligibles_classification}")
    print(f"   • Fichiers ≤ 2MB (→ AUTRES): {len(fichiers_info) - eligibles_classification}")
    
    return fichiers_info

def copier_fichiers(fichiers_info):
    """Phase 2: Copie vers Depots avec informations enrichies"""
    print(f"\n📋 PHASE 2: Copie vers {CHEMIN_DEPOTS}...")
    print("-" * 50)
    
    # Créer le dossier Depots
    os.makedirs(CHEMIN_DEPOTS, exist_ok=True)
    print(f"✅ Dossier Depots prêt")
    
    copied = 0
    taille_totale = 0
    
    for fichier_info in fichiers_info:
        file_path = fichier_info['chemin']
        file_name = fichier_info['nom']
        destination = os.path.join(CHEMIN_DEPOTS, file_name)
        
        try:
            shutil.copy(file_path, destination)
            copied += 1
            taille_totale += fichier_info['taille']
        except Exception as e:
            print(f"❌ Erreur copie {file_name}: {e}")
    
    print(f"✅ {copied}/{len(fichiers_info)} fichiers copiés")
    print(f"📦 Taille totale copiée: {taille_totale/(1024*1024):.1f} MB")
    return copied

def analyser_et_classer_semantique():
    """Phase 3: Analyse sémantique LLM et classification avancée (> 2MB seulement)"""
    print(f"\n🧠 PHASE 3: Analyse sémantique et classification...")
    print("-" * 50)
    
    # Mode LLM ou fallback
    use_llm = api_key != "votre_clé_api_openai_ici"
    print(f"Mode: {'🔥 LLM Sémantique' if use_llm else '🔧 Mots-clés enrichis'}")
    
    fichiers_a_analyser = [f for f in os.listdir(CHEMIN_DEPOTS) 
                          if f.lower().endswith(('.ppt', '.pptx', '.pdf'))]
    
    # Récupérer les infos de taille des fichiers copiés
    fichiers_avec_taille = []
    for file_name in fichiers_a_analyser:
        file_path = os.path.join(CHEMIN_DEPOTS, file_name)
        try:
            taille = os.path.getsize(file_path)
            fichiers_avec_taille.append({
                'nom': file_name,
                'taille': taille,
                'taille_mb': round(taille / (1024*1024), 2),
                'taille_kb': round(taille / 1024, 1),
                'eligible_classification': taille >= (3000 * 1024)  # > 2MB
            })
        except:
            # Si erreur, on considère comme petit fichier
            fichiers_avec_taille.append({
                'nom': file_name,
                'taille': 0,
                'taille_mb': 0,
                'taille_kb': 0,
                'eligible_classification': False
            })
    
    analyses = []
    medicaux = 0
    contextes_stats = {}
    petits_fichiers = 0
    
    print(f"📊 Répartition:")
    eligibles = sum(1 for f in fichiers_avec_taille if f['eligible_classification'])
    print(f"   • Fichiers > 2MB (analyse sémantique): {eligibles}")
    print(f"   • Fichiers ≤ 2MB (→ AUTRES direct): {len(fichiers_avec_taille) - eligibles}")
    print()
    
    for i, fichier_info in enumerate(fichiers_avec_taille, 1):
        file_name = fichier_info['nom']
        titre = os.path.splitext(file_name)[0]
        taille_str = f"{fichier_info['taille_kb']} KB" if fichier_info['taille_mb'] < 1 else f"{fichier_info['taille_mb']} MB"
        
        print(f"[{i:2d}/{len(fichiers_avec_taille)}] {file_name[:40]}... ({taille_str})")
        
        # Vérifier si éligible à la classification sémantique
        if not fichier_info['eligible_classification']:
            # Fichier ≤ 2MB → AUTRES directement
            analyse_simple = {
                'fichier': file_name,
                'maladies_detectees': [],
                'categorie_medicale': 'aucune',
                'contexte_principal': 'autre',
                'type_document': 'petit_fichier',
                'population_cible': 'aucune',
                'confiance': 'automatique',
                'contient_maladie': False,
                'score_semantique': 0.1,
                'justification': f'Fichier petit ({taille_str}) → AUTRES automatiquement',
                'taille_mb': fichier_info['taille_mb']
            }
            
            analyses.append(analyse_simple)
            petits_fichiers += 1
            print(f"     → AUTRES/ (taille: {taille_str})")
            
        else:
            # Fichier > 2MB → Analyse sémantique complète
            if use_llm:
                resultat = analyser_titre_avec_llm_semantique(titre)
            else:
                resultat = detection_mots_cles_medicaux(titre)
            
            # Enrichir l'analyse
            contexte = resultat.get('contexte_principal', 'autre')
            contextes_stats[contexte] = contextes_stats.get(contexte, 0) + 1
            
            analyse_complete = {
                'fichier': file_name,
                'maladies_detectees': resultat.get('maladies_detectees', []),
                'categorie_medicale': resultat.get('categorie_medicale', 'aucune'),
                'contexte_principal': contexte,
                'type_document': resultat.get('type_document', 'autre'),
                'population_cible': resultat.get('population_cible', 'aucune'),
                'confiance': resultat.get('confiance', 'faible'),
                'contient_maladie': resultat.get('contient_maladie', False),
                'score_semantique': resultat.get('score_semantique', 0.3),
                'justification': resultat.get('justification', 'Classification sémantique'),
                'taille_mb': fichier_info['taille_mb']
            }
            
            analyses.append(analyse_complete)
            
            if resultat.get("contient_maladie", False):
                medicaux += 1
                score = resultat.get('score_semantique', 0)
                maladies = '|'.join(resultat.get('maladies_detectees', []))
                print(f"     → {contexte.upper()}_{resultat.get('categorie_medicale', 'autre').upper()}/ ({maladies}) [score: {score:.2f}]")
            else:
                score = resultat.get('score_semantique', 0)
                print(f"     → {contexte.upper()}/ [score: {score:.2f}]")
        
        # Classer le fichier physiquement
        ancien_chemin = os.path.join(CHEMIN_DEPOTS, file_name)
        
        if fichier_info['eligible_classification'] and analyses[-1].get("contient_maladie", False):
            # Fichier médical > 2MB
            resultat = analyses[-1]
            categorie = resultat["categorie_medicale"]
            contexte = resultat["contexte_principal"]
            dossier_medical = os.path.join(CHEMIN_DEPOTS, f"{contexte.upper()}_{categorie.upper()}")
            os.makedirs(dossier_medical, exist_ok=True)
            nouveau_chemin = os.path.join(dossier_medical, file_name)
        elif fichier_info['eligible_classification']:
            # Fichier non-médical > 2MB
            resultat = analyses[-1]
            contexte = resultat["contexte_principal"]
            if contexte in ['commercial', 'formation', 'reglementaire']:
                dossier_contexte = os.path.join(CHEMIN_DEPOTS, f"{contexte.upper()}")
            else:
                dossier_contexte = os.path.join(CHEMIN_DEPOTS, "AUTRES")
            os.makedirs(dossier_contexte, exist_ok=True)
            nouveau_chemin = os.path.join(dossier_contexte, file_name)
        else:
            # Fichier ≤ 2MB → AUTRES
            dossier_autres = os.path.join(CHEMIN_DEPOTS, "AUTRES")
            os.makedirs(dossier_autres, exist_ok=True)
            nouveau_chemin = os.path.join(dossier_autres, file_name)
        
        # Déplacer
        try:
            shutil.move(ancien_chemin, nouveau_chemin)
        except Exception as e:
            print(f"     ❌ Erreur déplacement: {e}")
    
    print(f"\n📊 RÉSULTATS SÉMANTIQUES:")
    print(f"   • {medicaux}/{len(analyses)} fichiers médicaux classés")
    print(f"   • {petits_fichiers} petits fichiers (≤ 2MB) → AUTRES")
    print(f"   • {eligibles} fichiers analysés sémantiquement")
    if contextes_stats:
        print(f"   • Contextes détectés: {list(contextes_stats.keys())}")
        for ctx, count in contextes_stats.items():
            print(f"     - {ctx}: {count} fichiers")
    
    return analyses

def generer_rapport_semantique(analyses):
    """Phase 4: Génération du rapport avec analyse sémantique"""
    print(f"\n📝 PHASE 4: Génération du rapport sémantique...")
    print("-" * 50)
    
    fichiers_medicaux = sum(1 for a in analyses if a['contient_maladie'])
    categories = {}
    contextes = {}
    scores_moyens = {}
    
    for a in analyses:
        # Stats par catégorie médicale
        if a['contient_maladie']:
            cat = a['categorie_medicale']
            categories[cat] = categories.get(cat, 0) + 1
        
        # Stats par contexte
        ctx = a['contexte_principal']
        contextes[ctx] = contextes.get(ctx, 0) + 1
        
        # Scores sémantiques moyens
        if ctx not in scores_moyens:
            scores_moyens[ctx] = []
        scores_moyens[ctx].append(a.get('score_semantique', 0.3))
    
    # Calculer moyennes
    for ctx in scores_moyens:
        scores_moyens[ctx] = sum(scores_moyens[ctx]) / len(scores_moyens[ctx])
    
    rapport = f"""# 📊 Rapport d'Analyse Sémantique de Fichiers Médicaux

**Date d'analyse :** {datetime.now().strftime("%d/%m/%Y à %H:%M:%S")}  
**Dossier analysé :** `{CHEMIN_DEPOTS}`
**Méthode :** Analyse sémantique LLM + Classification contextuelle

---

## 📈 Résumé Statistique

| Métrique | Valeur |
|----------|--------|
| **Fichiers analysés** | {len(analyses)} |
| **Fichiers médicaux** | {fichiers_medicaux} |
| **Fichiers non-médicaux** | {len(analyses) - fichiers_medicaux} |
| **Catégories médicales** | {len(categories)} |
| **Contextes détectés** | {len(contextes)} |
| **Score sémantique moyen** | {sum(a.get('score_semantique', 0.3) for a in analyses) / len(analyses):.2f} |

---

## 🎯 Répartition par Contextes

"""
    
    for ctx, count in sorted(contextes.items()):
        pct = (count / len(analyses) * 100)
        score_ctx = scores_moyens.get(ctx, 0)
        rapport += f"- **{ctx.title()}** : {count} fichier(s) ({pct:.1f}%) - Score: {score_ctx:.2f}\n"
    
    rapport += "\n---\n\n## 🏥 Spécialités Médicales Détectées\n\n"
    
    if categories:
        for cat, count in sorted(categories.items()):
            pct = (count / fichiers_medicaux * 100) if fichiers_medicaux > 0 else 0
            rapport += f"- **{cat.title()}** : {count} fichier(s) ({pct:.1f}%)\n"
    else:
        rapport += "Aucune catégorie médicale détectée.\n"
    
    rapport += "\n---\n\n## 📋 Analyse Détaillée par Fichier\n\n"
    
    # Trier par score sémantique décroissant
    analyses_triees = sorted(analyses, key=lambda x: x.get('score_semantique', 0), reverse=True)
    
    for analyse in analyses_triees:
        contexte = analyse['contexte_principal']
        categorie = analyse['categorie_medicale']
        
        if analyse['contient_maladie']:
            dossier = f"{contexte.upper()}_{categorie.upper()}"
        else:
            dossier = contexte.upper() if contexte in ['commercial', 'formation', 'reglementaire'] else "AUTRES"
        
        score = analyse.get('score_semantique', 0)
        justification = analyse.get('justification', 'N/A')
        
        rapport += f"""### 📄 {analyse['fichier']}
- **Contexte :** {contexte}
- **Domaine médical :** {categorie}
- **Type document :** {analyse.get('type_document', 'N/A')}
- **Population cible :** {analyse.get('population_cible', 'N/A')}
- **Termes détectés :** {', '.join(analyse['maladies_detectees']) if analyse['maladies_detectees'] else 'Aucun'}
- **Score sémantique :** {score:.2f}/1.0
- **Confiance :** {analyse['confiance']}
- **Dossier :** `{dossier}/`
- **Justification :** {justification}

"""
    
    rapport += f"""---

## ⚙️ Technologie & Méthodologie

### Analyse Sémantique
- **LLM :** ChatOpenAI GPT-4o-mini (température: 0.1)
- **Approche :** Compréhension contextuelle vs simple recherche de mots-clés
- **Scoring :** Évaluation de cohérence sémantique (0.0 à 1.0)
- **Fallback :** Détection enrichie par mots-clés contextuels

### Classification Hiérarchique
- **Structure :** CONTEXTE_DOMAINE/ pour fichiers médicaux
- **Contextes :** {', '.join(sorted(contextes.keys()))}
- **Critères :** Finalité du document, population cible, type de contenu

### Métriques de Qualité
- **Confiance moyenne :** {sum(1 for a in analyses if a['confiance'] == 'haute') / len(analyses) * 100:.1f}% haute confiance
- **Classifications incertaines :** {sum(1 for a in analyses if a.get('score_semantique', 0) < 0.5)} fichiers (score < 0.5)

---

*Rapport généré par analyse sémantique LLM - Classification intelligente par contexte et intention 🧠*
"""
    
    # Sauvegarder
    chemin_rapport = os.path.join(CHEMIN_DEPOTS, "RAPPORT_SEMANTIQUE.md")
    with open(chemin_rapport, 'w', encoding='utf-8') as f:
        f.write(rapport)
    
    print(f"✅ Rapport sémantique sauvé: {chemin_rapport}")

def main():
    """Fonction principale - workflow sémantique simple"""
    print("🚀 CLASSIFICATEUR MÉDICAL SÉMANTIQUE")
    print("=" * 60)
    print(f"Source: {CHEMIN_SOURCE}")
    print(f"Destination: {CHEMIN_DEPOTS}")
    print("🧠 Analyse contextuelle et intentionnelle")
    print("=" * 60)
    
    # Workflow en 4 étapes avec analyse sémantique et filtrage
    fichiers_info = rechercher_fichiers_filtres()
    if not fichiers_info:
        print("❌ Aucun fichier trouvé après filtrage - Arrêt")
        return
    
    trier_fichiers_par_criteres(fichiers_info)
    copier_fichiers(fichiers_info)
    analyses = analyser_et_classer_semantique()
    generer_rapport_semantique(analyses)
    
    print("\n" + "🎉" * 20)
    print("✅ TERMINÉ - Classification sémantique réussie !")
    print("🧠 Analyse contextuelle vs mots-clés simples")
    print("📊 Structure hiérarchique par intention")
    print("🎉" * 20)

if __name__ == "__main__":
    main()